<a href="https://colab.research.google.com/github/SuryaVikram/TweetSentimentClassification_Flair/blob/master/Twitter_Sentiment_Analysis_Flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install the PyDrive wrapper & import libraries.


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '10zQ2a7SzVJik1vdSnzPKbINPrs7CT7Si' ### File ID ###
data = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))


In [9]:
import io
import pandas as pd
import re
import string
punctuation = string.punctuation
data = pd.read_csv(io.StringIO(data.GetContentString())) 
data.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [0]:
def _clean(text):
    text = text.lower()
    text = re.sub(r"amp", "", text)
    #text = re.sub(r"#\w+","",text)
    text = re.sub(r"@\w+","",text)
    
    
    text = "".join(x for x in text if x not in punctuation)

    L = [32, 44, 46, 65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,97,98,99,100,101,102,103, 104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122]
    text = str(text)

    return ''.join(i for i in text if ord(i) in L)

In [0]:
data['cleaned_tweet'] = data['tweet'].apply(_clean)

In [12]:
data.head()

,id,tweet,cleaned_tweet
0,31963,#studiolife #aislife #requires #passion #dedic...,studiolife aislife requires passion dedication...
1,31964,@user #white #supremacists want everyone to s...,white supremacists want everyone to see the ...
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe ways to heal your acne altwaystoheal h...
3,31966,is the hp and the cursed child book up for res...,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",rd bihday to my amazing hilarious nephew eli...


In [0]:
# download flair library #
# import torch
# !pip install flair
# import flair

In [14]:
from flair.data import Sentence
# create a sentence #
sentence = Sentence('Blogs of Analytics Vidhya are Awesome.')
# print the sentence to see what’s in it. #
print(Sentence) 

<class 'flair.data.Sentence'>


In [15]:
#extracting the tweet part#
text = data['cleaned_tweet'] 
 ## txt is a list of tweets ##
txt = text.tolist()
print(txt[:10])

['studiolife aislife requires passion dedication willpower   to find newmaterials ', '  white supremacists want everyone to see the new   birds movie  and heres why  ', 'safe ways to heal your acne    altwaystoheal healthy   healing ', 'is the hp and the cursed child book up for reservations already if yes where if no when    harrypotter pottermore favorite', '  rd bihday to my amazing hilarious nephew eli ahmir uncle dave loves you and misses ', 'choose to be    momtips ', 'something inside me dies   eyes ness smokeyeyes tired  lonely sof grunge ', 'finishedtattooinkedinkloveit  thanksaleeee  ', '    i will never understand why my dad left me when i was so young  deep inthefeels  ', 'delicious   food lovelife capetown mannaepicure resturant ']


In [0]:
## Importing the Embeddings ##
from flair.embeddings import WordEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings
from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings
from flair.embeddings import FlairEmbeddings

In [17]:
### Initialising embeddings (un-comment to use others) ###
#glove_embedding = WordEmbeddings('glove')
#character_embeddings = CharacterEmbeddings()
flair_forward  = FlairEmbeddings('news-forward-fast')
flair_backward = FlairEmbeddings('news-backward-fast')
#bert_embedding = BertEmbedding()
#elmo_embedding = ElmoEmbedding()

stacked_embeddings = StackedEmbeddings( embeddings = [ 
                                                       flair_forward, 
                                                       flair_backward
                                                      ])

2020-04-28 21:31:52,995 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmppeqv9vpz


100%|██████████| 19689779/19689779 [00:02<00:00, 8969948.75B/s]

2020-04-28 21:31:55,891 copying /tmp/tmppeqv9vpz to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2020-04-28 21:31:55,915 removing temp file /tmp/tmppeqv9vpz


In [31]:
# create a sentence #
sentence = Sentence("Analytics Vidhya blogs are Awesome .")
# embed words in sentence #
stacked_embeddings.embed(sentence)
for token in sentence:
  print(token)
  print(token.embedding.view(-1,))
# data type and size of embedding #
print(type(token.embedding))
# storing size (length) #
z = token.embedding.size()[0]
print(z)

Token: 1 Analytics
tensor([-2.6460e-02, -8.3260e-04, -3.1604e-03,  ...,  2.2739e-09,
        -2.2632e-08,  1.5214e-02])
Token: 2 Vidhya
tensor([-1.0809e-02, -7.1516e-05,  1.4545e-02,  ...,  5.8050e-09,
        -3.6731e-09,  6.9285e-02])
Token: 3 blogs
tensor([-8.2266e-03, -1.1461e-04,  1.4814e-02,  ...,  3.8778e-09,
         2.8023e-07, -9.3421e-02])
Token: 4 are
tensor([-1.1697e-02, -1.8518e-04,  1.9169e-02,  ...,  2.2128e-07,
         3.2774e-09, -2.6248e-01])
Token: 5 Awesome
tensor([-8.7913e-03, -5.9724e-04, -3.8459e-03,  ...,  7.3547e-09,
         5.5329e-09, -4.1644e-02])
Token: 6 .
tensor([ 2.7380e-04, -4.0641e-06,  1.8109e-03,  ...,  7.2283e-10,
         1.9171e-07, -1.5373e-02])
<class 'torch.Tensor'>
2048
